In [ ]:
### 
    # This code calculates the permafrost sensitivity estimated from Chadburn et al.(2017)
    # 1. Calculate average air temperature for each grid during 1982-2014 (ERA5-Land data)—— t2m_basemean
    # 2. Calculate the temperature change from 1986-2005 to 1982-2014 (HadCRUT5 data)
    # 3. Calculate temperature for each grid under three warming thresholds, using t2m_basemean and amplification factors
    # 4. Calculate permafrost probability corresponding to air temperature based on the formula
    # 5. Calculate permafrost sensitivity based on changes in permafrost area
    # 6. The sensitivity is plot in Figure 3.
###

In [ ]:
import pandas as pd
import numpy as np
import xarray as xr
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 1. Read NetCDF file
file_path = "/home/jidy/Data/ERA5-Land/ERA5-Land_2m-temperature_monthly_NH_1950-2022.nc"
ds = xr.open_dataset(file_path)

# 2. Extract variables and time information
t2m = ds["t2m"]
time = ds["time"]

# 3. Automatically parse time (priority: read from attributes)
time_units = ds["time"].attrs.get("units", None)
if time_units:
    # If time units exist, attempt to parse
    time_values = xr.cftime_range(start="1950-01", freq="MS", periods=len(time))
else:
    # Manually set time range
    time_values = xr.cftime_range(start="1950-01", periods=len(time), freq="MS")

t2m = t2m.assign_coords(time=("time", time_values))

# 4. Filter time range for 1982-2014
t2m_sel = t2m.sel(time=slice("1982-01", "2014-12"))

# 5. Replace missing values
fill_value = -32767
t2m = t2m.where(t2m != fill_value)

# 6. Decode scale_factor and add_offset
scale_factor = 0.00145277162700054
add_offset = 267.639219598073
t2m = t2m * scale_factor + add_offset

# 7. Calculate average temperature for each grid during 1982-2014
t2m_basemean = t2m_sel.mean(dim="time", skipna=True)

In [ ]:
def main():
    # Define two lists for the two time periods, storing annual average temperature anomalies for each year
    anomalies_1982_2014 = []
    anomalies_1986_2005 = []

    filename = "/home/wangjx/Data/HadCRUT5.0Analysis_gl.txt" 

    try:
        with open(filename, "r") as f:
            lines = f.readlines()
    except FileNotFoundError:
        print(f"File {filename} not found, please check if the path is correct.")
        return

    # Each year in the file has two lines of data, so the step size is 2
    for i in range(0, len(lines), 2):
        # First line: temperature anomaly data
        line_temp = lines[i].strip()
        if not line_temp:
            continue  # Skip empty lines

        # Split this line into several strings by whitespace characters
        tokens = line_temp.split()
        if len(tokens) < 14:
            # Incomplete data, skip
            continue

        try:
            year = int(tokens[0])
            # The annual average anomaly value is the last number in the line (total 13 numbers, with the first being the year, the next 12 being monthly data, and no annual value in between)
            # According to the file format description: format(i5,13f7.3), the actual data arrangement is: year, 12 months of data, annual data
            # The tokens list should have 1 + 13 = 14 elements, where tokens[-1] is the annual value.
            annual_anomaly = float(tokens[-1])
        except ValueError:
            # Data conversion failed, skip this line
            continue

        # If the year is between 1982 and 2014, store it in the corresponding list
        if 1982 <= year <= 2014:
            anomalies_1982_2014.append(annual_anomaly)
        # If the year is between 1986 and 2005, store it in the corresponding list
        if 1986 <= year <= 2005:
            anomalies_1986_2005.append(annual_anomaly)

    # Check if data was found
    if not anomalies_1982_2014:
        print("No data found for the 1982-2014 period.")
        return
    if not anomalies_1986_2005:
        print("No data found for the 1986-2005 period.")
        return

    # Calculate the average temperature anomalies for the two time periods
    avg_1982_2014 = sum(anomalies_1982_2014) / len(anomalies_1982_2014)
    avg_1986_2005 = sum(anomalies_1986_2005) / len(anomalies_1986_2005)

    # Calculate the temperature difference, representing the temperature change from 1986-2005 to 1982-2014
    delta = avg_1982_2014 - avg_1986_2005

    # Output results
    print("1982-2014 annual average temperature anomaly: {:.8f}".format(avg_1982_2014))
    print("1986-2005 annual average temperature anomaly: {:.8f}".format(avg_1986_2005))
    print("Compared to 1986-2005, the global average temperature in 1982-2014 increased by: {:.8f}".format(delta))

if __name__ == '__main__':
    main()

1982-2014 annual average temperature anomaly: 0.38363636
1986-2005 annual average temperature anomaly: 0.34915000
Compared to 1986-2005, the global average temperature in 1982-2014 increased by: 0.03448636


In [ ]:
degree = [1.5, 2, 3]
lat = ds["latitude"].data
lon = ds["longitude"].data
# Calculate average temperature relative to pre-industrial period
t2m_basemean = t2m_basemean - 273.15
for i in range(len(degree)):
    # Create an empty array with the same shape as taspre_mean
    tas_future = np.zeros_like(t2m_basemean)
    for j in range(len(lat)):
        if lat[j] <= 50:
            Amplification_factor = 2.319
        elif lat[j] <= 55:
            Amplification_factor = 2.484
        elif lat[j] <= 60:
            Amplification_factor = 2.600
        elif lat[j] <= 65:
            Amplification_factor = 2.482
        elif lat[j] <= 70:
            Amplification_factor = 2.474
        elif lat[j] <= 75:
            Amplification_factor = 2.552
        elif lat[j] <= 80:
            Amplification_factor = 2.254
        elif lat[j] <= 85:
            Amplification_factor = 1.797
        else:
            continue
        tas_future[j, :] = t2m_basemean[j, :] + (degree[i] - (0.61 + 0.03448636)) * Amplification_factor

    f = xr.Dataset(
        {'tas_future': (['lat', 'lon'], tas_future)},
        coords={'lat': lat, 'lon': lon}
    )
    # Add some metadata
    f.attrs['description'] = 't2m_basemean[j,:] + (degree[i] - (0.61 + 0.03448636)) * Amplification_factor'
    # Save as NetCDF file
    output_file = '../Data/Tas_used_in_Chadburn_approach/' + str(degree[i]) + '_tas_lat_lon.nc'
    f.to_netcdf(output_file)

In [ ]:
## Calculate historical period average temperature (lat, lon)
# 1. Read NetCDF file
file_path = "/home/jidy/Data/ERA5-Land/ERA5-Land_2m-temperature_monthly_NH_1950-2022.nc"
ds = xr.open_dataset(file_path)

# 2. Extract variables and time information
t2m = ds["t2m"]
time = ds["time"]

# 3. Automatically parse time (priority: read from attributes)
time_units = ds["time"].attrs.get("units", None)
if time_units:
    # If time units exist, attempt to parse
    time_values = xr.cftime_range(start="1950-01", freq="MS", periods=len(time))
else:
    # Manually set time range
    time_values = xr.cftime_range(start="1950-01", periods=len(time), freq="MS")

t2m = t2m.assign_coords(time=("time", time_values))

# 4. Filter time range for 1982-2014
t2m_sel = t2m.sel(time=slice("1982-01", "2014-12"))

# 5. Replace missing values
fill_value = -32767
t2m = t2m.where(t2m != fill_value)

# 6. Decode scale_factor and add_offset
scale_factor = 0.00145277162700054
add_offset = 267.639219598073
t2m = t2m * scale_factor + add_offset

# 7. Calculate average temperature for each grid during 1986-2005
t2m_basemean = t2m_sel.mean(dim="time", skipna=True)

# 8. Check results
print(t2m_basemean)

# 9. Save results as NetCDF file
lat = ds["latitude"].data
lon = ds["longitude"].data
f = xr.Dataset(
    {'tas_history': (['lat', 'lon'], t2m_basemean.data-273.15)},
    coords={'lat': lat, 'lon': lon}
)
# Add some metadata
f.attrs['description'] = '1982-2014_mean'
# Save as NetCDF file
output_file = '../Data/Chadburn_result/ERA5L_equilibrium_future_tas_lat_lon/his_tas_lat_lon.nc'
f.to_netcdf(output_file)

In [15]:
# Define constants
PI = np.pi
RADIUS = 6372 * 1e3  # Earth radius (m)
GRID_RES = 0.1  # Grid resolution (°)
MEAN = -4.38  # Mean value
SD = 2.59  # Standard deviation
FILL_VALUE = np.nan
pf = pd.DataFrame(columns=['his_pfarea', '1.5', '2', '3'])
# Read folder path for future temperature data
input_dir = "../Data/Tas_used_in_Chadburn_approach/"

# File name matching pattern
degree_list = ["his", "1.5", "2", "3"]  # List of degrees for looping

# Iterate through each file
for k in range(len(degree_list)):
    
    file_path = input_dir + degree_list[k] + '_tas_lat_lon.nc'
    print(f"Processing: {file_path}")
    # Read data
    ds = xr.open_dataset(file_path)
    lat = ds["lat"].data
    lon = ds["lon"].data
    if k == 0:
        tas = ds["tas_history"].data  # Replace with tas_future or tas_history
    else:
        tas = ds["tas_future"].data
        degree = degree_list[k]
    # Handle missing values
    # tas = np.where(np.isnan(tas), 0.0, tas)  # Replace NaN with 0
    print(f"TAS max: {np.nanmax(tas)}, TAS min: {np.nanmin(tas)}")
    
    # Calculate probability distribution P
    mean_arr = np.full_like(tas, MEAN)
    sd_arr = np.full_like(tas, SD)
    P = 1 - norm.cdf(tas, loc=mean_arr, scale=sd_arr)
    P = np.where(np.isnan(P), FILL_VALUE, P)  # Fill missing values
    print(f"P max: {np.nanmax(P)}, P min: {np.nanmin(P)}")
    # Calculate area weights
    nlat, nlon = len(lat), len(lon)
    areacella = np.zeros((nlat, nlon))
    for i in range(nlat):
        for j in range(nlon):
            areacella[i, j] = abs(
                RADIUS**2 * (GRID_RES / 180) * PI * (
                    np.sin((lat[i] + GRID_RES / 2) / 180 * PI) -
                    np.sin((lat[i] - GRID_RES / 2) / 180 * PI)
                )
            )

    # Masked P
    P_masked = P * areacella
    P_masked = np.where(np.isnan(P_masked), 0.0, P_masked)  # Handle missing values again

    # Calculate permafrost area
    pf_area = np.sum(P_masked) / 1e12  # Convert to million square kilometers
    if k == 0:
        pf.loc[0] = np.sum(P_masked[:451, 1801:]) / 1e12 + np.sum(P_masked[:451, :101]) / 1e12
    else:
        pf[str(degree)].loc[0] = np.sum(P_masked[:451, 1801:]) / 1e12 + np.sum(P_masked[:451, :101]) / 1e12
    print(f"Permafrost area: {pf_area} million km²")
    k = k + 1
pf

Processing: ../Data/Tas_used_in_Chadburn_approach/his_tas_lat_lon.nc
TAS max: 34.427215576171875, TAS min: -28.490447998046875
P max: 1.0, P min: 0.0
Permafrost area: 17.44524262936749 million km²
Processing: ../Data/Tas_used_in_Chadburn_approach/1.5_tas_lat_lon.nc
TAS max: 36.41115188598633, TAS min: -26.56212043762207
P max: 1.0, P min: 0.0
Permafrost area: 13.843132223792479 million km²
Processing: ../Data/Tas_used_in_Chadburn_approach/2_tas_lat_lon.nc
TAS max: 37.57065200805664, TAS min: -25.43511962890625
P max: 0.9999999999999998, P min: 0.0
Permafrost area: 11.86568500876187 million km²
Processing: ../Data/Tas_used_in_Chadburn_approach/3_tas_lat_lon.nc
TAS max: 39.889652252197266, TAS min: -23.181119918823242
P max: 0.9999999999998052, P min: 0.0
Permafrost area: 8.249233787163924 million km²


,his_pfarea,1.5,2,3
0,8.462535,6.604832,5.559088,3.557557


In [ ]:
# Calculate the permafrost sensitivity to global warming
pf_sen_percent = pd.DataFrame(columns = ['1.5','2','3'])
pf_sen_percent.loc[0] = (pf['1.5'].loc[0] - pf['his_pfarea'].loc[0]) / (1.5-(0.61+0.03448636))/pf['his_pfarea'].loc[0]*100
pf_sen_percent['2'].loc[0] = (pf['2'].loc[0] - pf['his_pfarea'].loc[0]) / (2-(0.61+0.03448636))/pf['his_pfarea'].loc[0]*100
pf_sen_percent['3'].loc[0] = (pf['3'].loc[0] - pf['his_pfarea'].loc[0]) / (3-(0.61+0.03448636))/pf['his_pfarea'].loc[0]*100
pf_sen_percent

,1.5,2,3
0,-25.659536,-25.311016,-24.606562
